In [105]:
import argparse 
import datetime
import random

import numpy as np
import tensorflow as tf


args = argparse.Namespace()
args.seed = 101
    
def random_date_output():
    ordinal_min = datetime.date(1000, 1, 1).toordinal()
    ordinal_max = datetime.date(9999, 12, 31).toordinal()
    return datetime.date.fromordinal(random.randint(ordinal_min, ordinal_max))

def output_date_to_input(date) -> str:
    months = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]
    
    return "{} {}, {}".format(months[date.month - 1], date.day, date.year)


def date_to_np(str_date):
    str_date = str_date.lower()
    return np.array([ord(s) for s in str_date])

def output_date_to_np(str_date):
    return np.array([10 if s=='-' else int(s) for s in str_date])

def np_to_date(np_date):
    return "".join([chr(i) for i in np_date.tolist()])

def build_dataset(dates_num):
    dataset = []
    input_list = []
    output_list = []
    for i in range(dates_num):
        rd = random_date_output()
        ird = output_date_to_input(rd)
        ird = date_to_np(ird)
        rd = output_date_to_np(str(rd))
        input_list.append(ird.tolist())
        output_list.append(rd.tolist())
        dataset.append([ird, rd])
    print(input_list)
    print(output_list)
    dataset = np.array(dataset, dtype=object)
    
    # return dataset[:, 0], dataset[:, 1]
    return tf.ragged.constant(input_list), tf.constant(output_list)


train_dataset = build_dataset(2)

max_int_len = ord("z")
max_out_len = len(train_dataset[1][0])
out_num = 11
print(max_out)
train_dataset
# np.vstack(train_dataset[1])

[[110, 111, 118, 101, 109, 98, 101, 114, 32, 49, 50, 44, 32, 53, 50, 54, 52], [110, 111, 118, 101, 109, 98, 101, 114, 32, 49, 44, 32, 53, 49, 49, 48]]
[[5, 2, 6, 4, 10, 1, 1, 10, 1, 2], [5, 1, 1, 0, 10, 1, 1, 10, 0, 1]]
10


(<tf.RaggedTensor [[110, 111, 118, 101, 109, 98, 101, 114, 32, 49, 50, 44, 32, 53, 50, 54, 52], [110, 111, 118, 101, 109, 98, 101, 114, 32, 49, 44, 32, 53, 49, 49, 48]]>,
 <tf.Tensor: shape=(2, 10), dtype=int32, numpy=
 array([[ 5,  2,  6,  4, 10,  1,  1, 10,  1,  2],
        [ 5,  1,  1,  0, 10,  1,  1, 10,  0,  1]], dtype=int32)>)

In [106]:
encoder = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(max_int, 32, input_shape=[None]),
    tf.keras.layers.LSTM(64, return_sequences=False)])

# RepeatVector is a must. we translate +- 17 words seq into 10 words seq. return_sequences = True for encoder leads up to 17 words output
# with return_sequences = False, it won't be possible to reconstruct a full seq, but one letter.
decoder = tf.keras.models.Sequential([
    tf.keras.layers.RepeatVector(max_out_len),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(out_num, activation="softmax")])
model = tf.keras.models.Sequential([encoder, decoder])

model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
model.summary()
model.fit(train_dataset[0], train_dataset[1])


# print(train_dataset[0][0])
# print(train_dataset[0][0].reshape(1, -1, 1).shape)
# model(train_dataset[0][0].reshape(1, -1, 1))

# print(train_dataset[0])
# print(train_dataset[0][0].reshape(2, -1, 1).shape)
# model(train_dataset[0])

Model: "sequential_73"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_71 (Sequential)  (None, 64)                28736     
                                                                 
 sequential_72 (Sequential)  (None, 10, 11)            33739     
                                                                 
Total params: 62,475
Trainable params: 62,475
Non-trainable params: 0
_________________________________________________________________


/Users/mkhokhlush/github/ml-experiments/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_73/sequential_71/lstm_51/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_73/sequential_71/lstm_51/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_73/sequential_71/lstm_51/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


1/1 [==============================] - 3s 3s/step - loss: 2.3970 - accuracy: 0.2000


<tf.Tensor: shape=(2, 10, 11), dtype=float32, numpy=
array([[[0.09115169, 0.0918684 , 0.09108949, 0.09062559, 0.09065845,
         0.09079225, 0.09082368, 0.09065931, 0.090624  , 0.09081312,
         0.090894  ],
        [0.09125575, 0.09252891, 0.09113512, 0.09044632, 0.09055454,
         0.09060355, 0.09082286, 0.0905228 , 0.09049722, 0.09082216,
         0.09081079],
        [0.09133481, 0.09303388, 0.09116194, 0.09028669, 0.0904934 ,
         0.09044617, 0.09081895, 0.09040909, 0.09042008, 0.09084263,
         0.09075238],
        [0.09139507, 0.0934168 , 0.0911772 , 0.09014739, 0.09046204,
         0.09031682, 0.09081183, 0.09031542, 0.09037658, 0.0908699 ,
         0.09071093],
        [0.09144101, 0.09370547, 0.09118547, 0.09002777, 0.09045064,
         0.09021161, 0.09080181, 0.09023909, 0.09035513, 0.09090067,
         0.09068134],
        [0.09147592, 0.09392219, 0.09118958, 0.0899264 , 0.09045199,
         0.0901266 , 0.09078941, 0.09017748, 0.09034756, 0.09093264,
         

In [104]:
for s in "abc0123, ":
    print(ord(s))
    
[[1, 2],[3, 4]][0][:]
np.array([np.array([1, 2]), np.array([3, 4])])


int("1")

97
98
99
48
49
50
51
44
32


1